
## Importing Modules


In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

## Loading the Dataset

In [ ]:
# Initialize lists to store file paths and labels
paths = []
labels = []

# Walk through the directory structure and collect file paths and labels
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        # Extract and preprocess labels from filenames
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        labels.append(label.lower())
    # Limit the number of files to 2800 for loading efficiency
    if len(paths) == 2800:
        break

# Print a message indicating that the dataset is loaded
print('Dataset is Loaded')

In [ ]:
# Display the first few file paths and labels
paths[:5]

In [ ]:
labels[:5]

In [ ]:
# Create a DataFrame to store file paths and labels
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels

# Display the first few rows of the DataFrame
df.head()

## Exploratory Data Analysis

In [ ]:
# Count and visualize the distribution of labels
df['label'].value_counts()
sns.countplot(data=df, x='label')


In [ ]:
# Define a function to plot the waveform of audio data
def waveplot(data, sr, emotion):
    plt.figure(figsize=(10, 4))
    plt.title(emotion, size=20)
    librosa.display.waveshow(data, sr=sr)
    plt.show()

# Define a function to plot the spectrogram of audio data
def spectrogram(data, sr, emotion):
    x = librosa.stft(data)
    xdb = librosa.amplitude_to_db(abs(x))
    plt.figure(figsize=(11, 4))
    plt.title(emotion, size=20)
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()


In [ ]:
emotion = 'fear'
path = np.array(df['speech'][df['label'] == emotion])[0]
data, sampling_rate = librosa.load(path)

waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)


In [ ]:
emotion = 'angry'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)


In [ ]:
emotion = 'disgust'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'neutral'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'sad'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)

In [ ]:
emotion = 'ps'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)


In [ ]:
emotion = 'happy'
path = np.array(df['speech'][df['label']==emotion])[0]
data, sampling_rate = librosa.load(path)
waveplot(data, sampling_rate, emotion)
spectrogram(data, sampling_rate, emotion)
Audio(path)

## Feature Extraction

In [ ]:
# Define a function to extract MFCC features from an audio file
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc

In [ ]:
# Example: Extracting MFCC features for the first file
extract_mfcc(df['speech'][0])

In [ ]:
# Apply MFCC extraction to all audio files in the DataFrame
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))
X_mfcc

In [ ]:
# Convert the MFCC data to a numpy array
X = [x for x in X_mfcc]
X = np.array(X)

In [ ]:
# Check the shape of the MFCC data
X.shape

In [ ]:
# Reshape the data to add a channel dimension
X = np.expand_dims(X, -1)
X.shape

In [ ]:
# Perform one-hot encoding for labels
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])
y = y.toarray()
y.shape

## Creating the LSTM Model

In [ ]:
# Import Keras libraries for building the neural network model
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout


In [ ]:
# Define the neural network model
model = Sequential([
    LSTM(256, return_sequences=False, input_shape=(40, 1)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(X, y, validation_split=0.2, epochs=50, batch_size=64)

## Plot the results

In [ ]:
# Plot training history - accuracy
epochs = list(range(50))
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, label='train accuracy')
plt.plot(epochs, val_acc, label='val accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot training history - loss
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(epochs, loss, label='train loss')
plt.plot(epochs, val_loss, label='val loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

## Building a Predictive System

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
# Reshape the data to remove the channel dimension
X = np.squeeze(X, axis=-1)

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize and train the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [ ]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Generate a classification report
class_report = classification_report(y_test, y_pred)
print("Classification Report:\n", class_report)


In [ ]:
# Function to predict emotion from a new audio file
def predict_emotion(audio_file_path):
    y, sr = librosa.load(audio_file_path, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    predicted_emotion = rf_classifier.predict([mfcc])  # Pass a 2D array with one sample
    return predicted_emotion[0]

# Example usage:
# Replace 'your_audio_file.wav' with the path to your audio file
# predicted_emotion = predict_emotion('your_audio_file.wav')
# print(f"Predicted Emotion: {predicted_emotion}")